# DART 오픈 API 01 - 개요와 활용

- DART 오픈AP 사용법
- json_normalize() 활용 
- 기업개황 API
- 정보검색 API
- 개별 보고서와 하위 보고서 URL 얻기
- 첨부파일과 재무제표 엑셀 파일 가져오기

### 오픈 API

- 공시 보고서 목록를 실시간 제공
- https://dart.fss.or.kr/dsap001/intro.do 오픈API 소개
- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

### API 인증키 발급

- 계정을 만들고, API 인증키 발급 받는다

- 계정 신청: https://dart.fss.or.kr/dsag002/insertForm.do

### 개인용, 기업용
- 개인용: 1만건/일
- 기업용: 조회 건수 제한이 없는 대신 조회 서버 IP 제한

### 인증키
- 알파벳과 숫자로 이루어진 40 바이트 문자열
- auth 파라미터로 제공

http://dart.fss.or.kr/api/search.xml?auth=e81e78aa519d1a1c4ec7867ef12a937ab6cbd4c7

### 오픈 API가 제공하는 주요정보

 - 주요하게 다음 2가지 API를 제공한다
    1. 기업 개황 API : http://dart.fss.or.kr/api/company.json?auth=xxx&crp_cd=xxx
    2. 정보 검색 API : http://dart.fss.or.kr/api/search.json?auth=xxx



## API 개발 가이드 사이트에서 테스트

- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

![](http://i.imgur.com/9NpoJgx.png)